<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/PairwiseSumo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 強さ判定

出典：https://github.com/lucasmaystre/choix/blob/master/notebooks/intro-pairwise.ipynb

今日やること：不揃いの対戦結果から、強さのランキングを計算します。--- Bradely-Terry モデル 

## 準備

In [ ]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org choix networkx lxml html5lib bs4 japanize-matplotlib pydot
#!pip install choix networkx lxml html5lib bs4 japanize-matplotlib

# 一対比較による強さ判定

ここでは `choix` ライブラリを使います。一対一の対戦結果のデータがあるとします。

In [ ]:
import choix
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline
np.set_printoptions(precision=3, suppress=True)

`choix` では、$n$ 個の要素に $\{0, \ldots, n-1 \}$ の番号を付けます。
"$i$ が $j$ に勝った" ことを Python のタプル `(i, j)` で表します（左が勝者、右が敗者）。

簡単な例で動作確認をしてみます。

In [ ]:
n_items = 5
data = [
    (1, 0), (0, 4), (3, 1),
    (0, 2), (2, 4), (4, 3),
]

このデータをグラフとして可視化します。

- 各要素はグラフのノードに対応します。
- ノード $i$ からノード $j$ へのリンクは、"$i$ が $j$ に勝った"を示します。

In [ ]:
graph = nx.DiGraph(data)
nx.draw(graph, with_labels=True)
plt.show()

このデータを、 [Bradley-Terry model](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model) によって解析します。
`choix` はこのためのアルゴリズムを用意しています。以下では、 I-LSR と呼ばれる最尤推定法を用います。

In [ ]:
params = choix.ilsr_pairwise(n_items, data)
print(params)

パラメータ (params) は各要素の"強さ" (or utility) を表します。パラメータの値でソートすることで、強さのランキングを算出できます。

In [ ]:
print("強さランキング:", np.argsort( -params ))

## スパース性の取り扱い
グラフが連結していないと、最尤推定法が使えません。このような状況は、全勝もしは全敗の要素がある時に生じます。以下の例では、$3$ が全勝、$1$ が全敗です。

In [ ]:
n_items = 4
data = [(3, 2), (2, 1), (1, 0)]

graph = nx.DiGraph(data)
nx.draw(graph, with_labels=True)

このようなケースでは、予測に失敗します。

In [ ]:
choix.ilsr_pairwise(n_items, data)

この問題は、正規化パラメータの導入により解決できます。

In [ ]:
choix.ilsr_pairwise(n_items, data, alpha=0.01)

# 課題　大相撲で今一番強いのは？

先場所の結果から、Bradely-Terry モデルによって現時点での強さランキングを出してみましょう。強さが番付とどれくらい一致しているかを調べましょう。

In [ ]:
from bs4 import BeautifulSoup
import requests
import itertools
import choix
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import time
import japanize_matplotlib
%matplotlib inline

例えば2022年秋場所の初日を例とすると、

In [ ]:
response = requests.get("https://sports.yahoo.co.jp/sumo/torikumi/201009/1", verify=False)
html_doc = response.text
soup = BeautifulSoup(html_doc, 'html.parser')
print(soup.prettify())

これを見ると、対戦結果は tr タグの su-scoraTable__row クラスに入っています。さらにその中を見ると、名前は、p タグの su-scoreTable__nameDetail クラスに、勝敗は span タグの aria-label に入っています。

初日から千秋楽までの分をまとめて取得します。

In [ ]:
results = []
for i in range(1,16):
    print(str(i) + "日目のデータ取得")
    response = requests.get("https://sports.yahoo.co.jp/sumo/torikumi/202209/"+str(i), verify=False)
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    results.append(soup.find_all("tr",{"class":"su-scoreTable__row"}))
    time.sleep(1)
results

In [ ]:
data = []
for tags in results: # １５日分繰り返す
    for i in tags: # １日の試合数分繰り返す
        names = i.find_all("p", {"class":"su-scoreTable__nameDetail"})
        name1 = names[0].string
        name2 = names[1].string
        result = i.find_all("span")
        if "勝利" in str(result[0]):
            data.append([name1, name2])
        else:
            data.append([name2, name1])
data

In [ ]:
import matplotlib.pyplot as plt
graph = nx.DiGraph(data)
pos = nx.spring_layout(graph)
nx.draw_networkx(graph, pos, with_labels=True, node_shape=".", node_size=30, font_size=8, font_family='Osaka')

勝敗リストを平坦化し、重複を取り除くと名前のリストになります。

In [ ]:
names = list(set(itertools.chain.from_iterable(data)))
len(names)

In [ ]:
data2 = []
for i in data:
    #print(names.index(i[0]), ",", names.index(i[1]))
    data2.append([names.index(i[0]), names.index(i[1])])

In [ ]:
params = choix.ilsr_pairwise(len(names), data2, alpha=0.1)
params

In [ ]:
print("強さランキング:", np.argsort( -params ))

In [ ]:
ranking = np.argsort( -params )
for i in ranking:
    print(names[i])

In [ ]:
graph = nx.DiGraph(data)
nx.draw(graph, with_labels=True, font_family='Osaka')

## 課題：最強は誰か？
時期を決めて最強は誰かを見つけてください（例：2010年代最強、平成最強、令和最強）
